In [2]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## Randomized Algorithms - Selection


#### Selection: Finding the $k$-th minimal element given a sequence

Given an unsorted list $a$ and an integer $k$ ($0\leq k< |a|$), the **order statistics** (or **selection**) problem asks us to return the $k$-th smallest element from $a$. 

>We also refer to the $k$-th smallest element as the element of *rank* $k$.  

<span style="color:orange">Example</span>: Let $a=\langle 2, 5, 4, 1, 3, -1, 99\rangle.$ For $k=0$, the "$0$-th smallest" element is the minimum element in $a$, or $-1$. For $k=n-1$, it is the maximum, or $99$. For $k=3$, we return $3$.


<span style="color:green">**Brute Force**??</span> Search Space, Work/Span?


<span style="color:red">**Reduction**??</span> lower bound and upper bound? 



**Problem A**: Finding the $k$-th minimal element<br><br>

**Problem B**: Sorting<br>
**Problem C**: Finding the minimal element


First, any algorithm for this problem requires $\Omega(n)$ work. 

<span style="color:blue">Quiz:</span> $\Omega(n), \Theta(n), \mathcal{O}(n)$

> Check $f(n) = n^2, ~~~g(n) = \sqrt(n), ~~~h(n) = \log^2 n$

Second, we can reduce this problem to sorting: we sort $a$ and return the $k$-th element of this sorted list. This requires $O(n\log n)$ time. 


Can we do any better? Sorting seems like overkill since we don't really need to rearrange all the elements, or even return a list.

A useful observation is that the $k$-th smallest element in the list *partitions* the list into:
- a set of $k$ smaller elements, 
- a set of $n$-$k$-$1$ larger elements. 

Example: Suppose $a=\langle 2, 5, 4, 1, 3, -1, 99\rangle$ and $k=3$. So $3$ is larger than $\langle 2, 1, -1 \rangle$ and smaller than $\langle5, 4, 99\rangle$.



Notice that *for any* element $x$ in the list, we can look at each element in the list to compute the rank of $x$. This can be done in $O(n)$ work and $O(\log n)$ span. 






<br>
<br>
<br>

<span style="color:Blue">Example</span>: Suppose $a=\langle 2, 5, 4, 1, 3, -1, 99\rangle$ and $k=3$. 

For $a[0] = 2$, it is larger than 2 elements ($\langle 1, -1 \rangle$) and smaller than 5 elements ($\langle 5, 4, 3, 99\rangle$).


<br><br>
We can see from this example that once we've identified 3 smaller elements ($\langle 1, -1 \rangle$ and $2$), the element of rank $k=3$ in $a$ must be in $\langle5, 4, 3, 99\rangle$. Moreover, it's new rank is $k-3 = 0$ in this list. 

This is a little like binary search, but with the partition step helping establish some order.

This observation motivates the following recursive algorithm:

<p>\begin{array}{ll}  
\mathit{simple\_select}~a~k =   
\\  
~~\texttt{let}  
\\  
~~~~p = a[0]   
\\  
~~~~\ell = \left\langle\, x \in a \;|\; x < p \,\right\rangle  
\\  
~~~~r = \left\langle\, x \in a \;|\; x > p \,\right\rangle  
\\  
~~\texttt{in}  
\\  
~~~~\texttt{if}~(k < |\ell|)~\texttt{then}~\mathit{simple\_select}~\ell~k  
\\  
~~~~\texttt{else if}~(k < |a| - |r|)~\texttt{then}~p  
\\  
~~~~\texttt{else}~\mathit{simple\_select}~r~(k - (|a| - |r|))  
\\  
~~\texttt{end}  
\end{array}</p>


In [1]:
def simple_select(a, k):
    p = a[0]
    print('\na=', a, 'k=', k, 'p=', p)
    l = list(filter(lambda x: x < p, a))  
    r = list(filter(lambda x: x > p, a))  
    print('l=', l, 'r=', r)
    if k < len(l):
        print('...recursing with l=%s and k=%d' % (str(l), k))
        return simple_select(l, k)
    elif k < len(a) - len(r):
        print('...returning p=%d' % p)
        return p
    else:
        print('...recursing with r=%s and k=%d' % (str(r), k - (len(a) -  len(r))))
        return simple_select(r, k - (len(a) -  len(r)))
    
# -1, 1, 2, 3, 4, 5, 99
# k=3 -> 3
# k=0 -> -1
# k=6 -> 99
simple_select([2,5,4,1,3,-1,99], 3)


a= [2, 5, 4, 1, 3, -1, 99] k= 3 p= 2
l= [1, -1] r= [5, 4, 3, 99]
...recursing with r=[5, 4, 3, 99] and k=0

a= [5, 4, 3, 99] k= 0 p= 5
l= [4, 3] r= [99]
...recursing with l=[4, 3] and k=0

a= [4, 3] k= 0 p= 4
l= [3] r= []
...recursing with l=[3] and k=0

a= [3] k= 0 p= 3
l= [] r= []
...returning p=3


3


We just have one recursive call so no parallelism there. However, we can use filter to partition in parallel. This has $O(n)$ work $O(\log n)$ span.

What is the total work over all recursions? We know that the work in each recursive call is 
$$\max\{W(\mid l\mid), W(\mid r\mid)\} + O(n)$$

<span style="color:blue">What is the worst case?? Is this divide-and-conquer or not?</span>

Consider the case where $a$ is a sorted list. Then in every call, $\ell = \emptyset$, and $\mid r\mid = n-1$. So we would we have $W(n) = W(n-1) + n = O(n^2)$. This is worse than just sorting the list!

Moreover the span is $S(n) = S(n-1) + \lg n \in O(n\lg n)$ - far worse than sorting $O(\lg^2 n)$!



#### What can we do?

<img width="60%" src="hope_pray.jpeg"/>


### Randomized Algorithms

When we don't know exactly how to proceed in solving a problem, we can make a random choice and hope we made the right one.

Alternately, we can view randomization as helping us avoid always making the worst/wrong choice. 

Additionally for instances where we might not have a good way to proceed, we can "guess" an answer and "hope" it's a good choice. 

What does "hope" mean, computationally? A proof that at least most of the time, we are correct/efficient.

<br><br>
Randomized algorithms can often be:

1- easier to understand

2- faster

3- more robust to adversarial input (cryptography)

The problem is that we just don't know anything about the element we're using for the partition. How do we avoid this worst case?

Pick a random element, or **pivot**, for partitioning!

<p>\begin{array}{ll}  
\mathit{select}~a~k =   
\\  
~~\texttt{let}  
\\  
~~~~p = \mbox{pick a uniformly random element from}~a   
\\  
~~~~\ell = \left\langle\, x \in a \;|\; x < p \,\right\rangle  
\\  
~~~~r = \left\langle\, x \in a \;|\; x > p \,\right\rangle  
\\  
~~\texttt{in}  
\\  
~~~~\texttt{if}~(k < |\ell|)~\texttt{then}~\mathit{select}~\ell~k  
\\  
~~~~\texttt{else if}~(k < |a| - |r|)~\texttt{then}~p  
\\  
~~~~\texttt{else}~\mathit{select}~r~(k - (|a| - |r|))  
\\  
~~\texttt{end}  
\end{array}</p>


In [5]:
import random
# random.seed(42)  # for repeatability

def select(a, k):
    p = random.choice(a)
    print('\na=', a, 'k=', k, 'p=', p)
    l = list(filter(lambda x: x < p, a))  # O(|a|) work, O(log|a|) span
    r = list(filter(lambda x: x > p, a))  # O(|a|) work, O(log|a|) span
    print('l=', l, 'r=', r)
    if k < len(l):
        print('...recursing with l=%s and k=%d' % (str(l), k))
        return select(l, k)
    elif k < len(a) - len(r):
        print('...returning p=%d' % p)
        return p
    else:
        print('...recursing with r=%s and k=%d' % (str(r), k - (len(a) -  len(r))))
        return select(r, k - (len(a) -  len(r)))
    
select([2,5,4,1,3,-1,99], 3)


a= [2, 5, 4, 1, 3, -1, 99] k= 3 p= -1
l= [] r= [2, 5, 4, 1, 3, 99]
...recursing with r=[2, 5, 4, 1, 3, 99] and k=2

a= [2, 5, 4, 1, 3, 99] k= 2 p= 5
l= [2, 4, 1, 3] r= [99]
...recursing with l=[2, 4, 1, 3] and k=2

a= [2, 4, 1, 3] k= 2 p= 3
l= [2, 1] r= [4]
...returning p=3


3


If we get a sorted list as input, what is the probability of the worst-case?

The size of the $l$ and $r$ will depend on the random choice. Thus, the recurrences describing the work and span depend on each random choice and we need to find their **expected asymptotic work/span**.

### Expected Work and Span

**Expectation** of a random variable $X$ defined for a probability space $(\Omega, \mathbf{P})$ is denoted ($\Omega$ is the sample space):

$$ \mathbf{E}\left[{X}\right] = \sum_{x} x \cdot \mathbf{P}\left[X = x\right]. $$



``Example 1``: Let $X$ be the sum value of **a pair of dice**. Each trial is independent. 

$X$ can take on values between 2 and 12, where each has a certain probability based on the possible ways the dice can sum to that value.  What is the expected value of the value of a pair of unbiased dice? 


$\begin{eqnarray*}
\mathbf{E}\left[{X}\right] &=& \sum_{x=2}^{12} x \cdot \mathbf{P}\left[X = x\right] \\
&=& 2\cdot\frac{1}{36} + 3\cdot\frac{2}{36} + 4\cdot\frac{3}{36} + \cdots + 11\cdot\frac{2}{36} + 12\cdot\frac{1}{36}\\
& = & 7
\end{eqnarray*}$

#### One More Example: Randomly filtering elements

Suppose we are given a list $L$ of length $n$. 

For each element we flip an unbiased coin $x_i$. Return a list $R$ with $L[i]$ such that $x_i$ is heads.

What is the expected size of $R$?


Let $X_i$ be an indicator random variable that is 1 if element $i$ is chosen, and 0 otherwise. The size of $R$ could be formulated as $X = \sum_{i=0}^{n-1} X_i$. So by **linearity of expectation**, we can compute:

$$\mathbf{E}[X] = \sum_{i=0}^{n-1} \mathbf{E}[X_i].$$

Now notice that by the definition of expectation 

$$E[X_i] = 0\cdot \mathbf{P}[X_i = 0] + 1\cdot\mathbf{P}[X_i = 1] = \mathbf{P}[X_i = 1].$$

Since we flip a coin for each element independently, $\mathbf{P}[X_i = 1] = 1/2.$ Thus, $\mathbf{E}[X_i] = 1/2$ for all $i$. So we have that

$$\mathbf{E}[X] = \sum_{i=0}^{n-1} \frac{1}{2} = \frac{n}{2}.$$


So the expected size of $R$ is $n/2$.

How about considering the expected size directly?

Let $X$ be the size of $R$. What is the possible value of $X$? What is the probability per each value?

<br>
<br>
<br>

$X$ can be from 0 to $n$

What is $P[X]$?

<br>
<br>
<br>
<br>

$$\mathbf{E}[X] = 0\frac{C_n^0}{2^n}+1\frac{C_n^1}{2^n}+2\frac{C_n^2}{2^n}+\cdots + n\frac{C_n^n}{2^n} = \frac{n2^{n-1}}{2^n} = \frac{n}{2} $$

``Example 3``: Let's look at coin flips where we have equal probability of heads/tails, and define $X$ as the number of flips until we get heads. What is $E[X]$? 

Observe that:

$\begin{eqnarray*}
E[X] &=& \frac{1}{2}\times{1} + \frac{1}{2}\times(1+E[X]) \\
\end{eqnarray*}$

Solving for $E[X]$, we get that $E[X] = 2$. 



One more solution:

$\begin{eqnarray*}
E[X] &=& \frac{1}{2}\times{1} + \Big(\frac{1}{2}\Big)^2\times{2}+\Big(\frac{1}{2}\Big)^3\times{3}+\cdots\Big(\frac{1}{2}\Big)^i\times{i} ~~~(i\rightarrow \infty)\\
& = & \sum_{i=1}^{\infty}\Big(\frac{1}{2}\Big)^i\times{i}\\
& = & 2\\
\end{eqnarray*}$

More generally, the expected number of trials to get an outcome of probability $p$ is $1/p$. (We'll see this later.)


When using randomization we usually have one of two types of algorithms.

A **Monte Carlo** randomized algorithm has a **deterministic** worst-case runtime but a randomized output that is correct with some probability.

A **Las Vegas** randomized algorithm always produces a correct solution, but has an **expected** runtime.  

We will focus on Las Vegas algorithms.

<p>\begin{array}{ll}  
\mathit{select}~a~k =   
\\  
~~\texttt{let}  
\\  
~~~~p = \mbox{pick a uniformly random element from}~a   
\\  
~~~~\ell = \left\langle\, x \in a \;|\; x < p \,\right\rangle  
\\  
~~~~r = \left\langle\, x \in a \;|\; x > p \,\right\rangle  
\\  
~~\texttt{in}  
\\  
~~~~\texttt{if}~(k < |\ell|)~\texttt{then}~\mathit{select}~\ell~k  
\\  
~~~~\texttt{else if}~(k < |a| - |r|)~\texttt{then}~p  
\\  
~~~~\texttt{else}~\mathit{select}~r~(k - (|a| - |r|))  
\\  
~~\texttt{end}  
\end{array}</p>


Let's get some intution for what's happening. We saw that the work of our algorithm depends on $\max\{W(\mid l\mid), W(\mid r\mid)\}$ in each recursive call. While there is only a $1/n$ probability of choosing a balanced split, any constant fraction reduction in the size of the larger list yields good performance. 

Let $X(n)$ be the fractional size of the larger side of the split, for an input of size $n$. So 

$$X(n) = \frac{\max{\{|l|, |r|\}}}{n}$$

e.g., $n=6$

|i  | len(l) | len(r)  | X(i) |
|---|--------|---------|------|
|0  | 0      | 5       | 5/6  |
|1  | 1      | 4       | 4/6  |
|2  | 2      | 3       | 3/6  |
|3  | 3      | 2       | 3/6  |
|4  | 4      | 1       | 4/6  |
|5  | 5      | 0       | 5/6  |

Then the work and span recurrences are:

$$W(n) \leq W(X(n) \cdot n) + O(n)$$

$$S(n) \leq S(X(n) \cdot n) + O(\lg n)$$ 

<br>

First, we'll bound $\mathbf{E}[X(n)].$ If $|l| = i$, then $|r| = n - i -1$. Using the fact that the probability of the pivot being any particular $i$ is $1/n$, we have:

$$\begin{eqnarray*}
 \mathbf{E}\left[{X(n)}\right] &=&  \sum_{i=0}^{n-1}P[X(i)] \cdot X(i)\\
 & = & \frac{1}{n} \sum_{i=0}^{n-1} \frac{\max\{i, n-i-1\}}{n} \\
 &\leq & \frac{1}{n} \sum_{j=n/2}^{n-1} \frac{2}{n} \cdot j  \\
 & \leq & \frac{2}{n^2}\sum_{j=n/2}^{n-1} j =\frac{2}{n^2}\times\frac{3n^2 - 2n}{8}\\
 &\leq & \frac{3}{4}  
\end{eqnarray*}$$

<br>



$$W(n) \leq W(\frac{3}{4}n) + O(n) \Rightarrow \mathrm{root~dominated}  \Rightarrow W(n)=\mathcal{O}(n)$$

$$S(n) \leq S(\frac{3}{4}n) + O(\lg n) \Rightarrow \mathrm{balanced} \Rightarrow S(n) = \mathcal{O}(\lg^2n)$$ 


It might seem tempting to say that we are done. However, we could get "unlucky" in a series of recursions even though $\mathbf{E}[X(n)]\leq 3/4.$ 

What we want to know is:  **What's the probability that the input to the recursive call has size $\le \frac{3}{4} n$ ?**
<br>

We can examine where $p$ might land in the sorted version of $a$, to understand the probability of a good split.

<img width="60%" src="selection-intuition.jpg"/>

If the sampled pivot lies in the green region, then the size of the array passed to the recursive call is at most $3n/4$.

The probability of sampling a point in the green region is $1/2$.

We can see that $\mathbf{P}[\max\{W(\mid l\mid), W(\mid r\mid)\} \leq W(3n/4)] = 1/2$.



If we think of each choice of pivot as a coin flip ("good" vs. "bad") then the expected number of pivot choices to reduce the input to $3n/4$ is 2. 

In other words, every two recursions yield the desired reduction in list size, and so in expectation we will do linear work.

<br><br>
**What if we're unlucky?**

We could keep sampling pivots outside of the green area. What is the probability if we do so $i$ times in a row?

<br><br>
$\frac{1}{2} * \frac{1}{2} * \frac{1}{2} * ... = \frac{1}{2^i}$

E.g., for $i=10$, probability of getting no good pivots is $\approx 0.1\%$. 

Thus, probability of getting at least one good pivot for 10 splits is $\approx 99.9\%$